<a href="https://colab.research.google.com/github/ProtossDragoon/CoMoLab/blob/master/CV/Darknet/YOLOv2in_Colab_janghoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google Drive 연결

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

## Install linux dependencies to colab session

In [0]:
!apt-get update
!apt-get upgrade

!apt-get install build-essential
!apt-get install cmake git libgtk2.0-dev pkg-config libavcodec-dev libavformat-dev libswscale-dev

!apt-get install libavcodec-dev libavformat-dev libswscale-d

!apt-get install libopencv-dev

!apt install g++-5
!apt install gcc-5

!apt update
!apt upgrade

## [only once] Install Darknet

In [0]:
# 구글 드라이브에 darknet을 설치
# version : AlexeyAB : https://github.com/AlexeyAB/darknet

%cd /content/gdrive/"My Drive"
# !git clone https://github.com/AlexeyAB/darknet

%cd darknet
!ls

!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!make

## [only once] Download pretrained weight

### [only once] yolov3 weight for DEMO

In [0]:
%cd /content/gdrive/"My Drive"/darknet
!wget https://pjreddie.com/media/files/yolov3.weights

## Run Demo

### Run Demo Image

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)
print("'sup!'")

!/usr/local/cuda/bin/nvcc --version

# 디렉터리 이동
%cd /content/gdrive/"My Drive"/darknet
!rm predictions.jpg

# 실행권한 부여
!chmod u+x ./darknet

# 실행 명령
!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg

def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
  
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)
  
# 실행의 결과물 파일을 보여줌
imShow('predictions.jpg')

# Train Custom Dataset


https://github.com/AlexeyAB/darknet/tree/47c7af1cea5bbdedf1184963355e6418cb8b1b4f#how-to-train-to-detect-your-custom-objects


## Training model : tiny-yolo v2

pre-trained weights for convolution layers

### [only once] Prepare Gun Dataset

In [0]:
# Dwonload / Upload dataset
# Gun dataset settting
# upload gun dataset 'zip file' to google colab

%cd /content/gdrive/"My Drive"/darknet/data/customdata
!rm -r gundata
!mkdir gundata

!unzip images_and_labels.zip -d /content/gdrive/"My Drive"/darknet/data/customdata/gundata
!ls

### [only once] Change .cfg file

In [0]:
# 1. copy yolov2-tiny-voc.cfg file
# 2. rename yolov2-tiny-voc 사본.cfg to yolov2-tiny-voc-gundetection.cfg
# 3. yolov2-tiny-voc-gundetection.cfg 수정

## 수정 내용 :

### [net] 부분
'''
[net]
# Testing
# batch=1
# subdivisions=1
# Training
batch=64
subdivisions=2
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.0007
max_batches = 10000
policy=steps
steps=-1,100,20000,30000
scales=.1,10,.1,.1
'''

### 거의 마지막 부분
'''
###########

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters=30
activation=linear

[region]
anchors = 1.08,1.19,  3.42,4.41,  6.63,11.38,  9.42,5.11,  16.62,10.52
bias_match=1
classes=1
coords=4
num=5
softmax=1
jitter=.2
rescore=1

object_scale=5
noobject_scale=1
class_scale=1
coord_scale=1

absolute=1
thresh = .3
random=1
'''

### [only once] Get pretrained weight

In [0]:
# Download default weights file for tiny-yolo-voc: http://pjreddie.com/media/files/tiny-yolo-voc.weights
# but not work, so find yourself!

# download from here! https://pjreddie.com/darknet/yolov2/

%cd /content/gdrive/"My Drive"/darknet/weights

In [0]:
# darknet partial cfg/tiny-yolo-voc.cfg tiny-yolo-voc.weights tiny-yolo-voc.conv.13 13
# 이 명령어를 수행하면, tiny-yolo-voc.weights 에서 cfg/tiny-yolo-voc.cfg 의 13번째 conv layer 까지만의 정보를 담고 있는
# 새로운 가중치 파일을 생성함. 

# 실행권한 부여
!chmod u+x ./darknet

%cd /content/gdrive/"My Drive"/darknet
!./darknet partial cfg/tiny-yolo-voc.cfg weights/yolov2-tiny-voc.weights weights/yolov2-tiny-voc.conv.13 13

/content/gdrive/My Drive/darknet
 OpenCV isn't used 
batch = 8, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256       3 x 3/ 1     26 x  26 x 128 ->   26 x  26 x 256 0.399 BF
   9 max                2x 2/ 2     26 x  26 x 256 ->   13 x  13 x 256 0.000 BF
  10 conv    512       3 x 3/ 1     13 x 

### [only once] Prepare filename text

In [0]:
# prepare these files yourself !

## obj-gundetection.data
## obj-gundetection.names
## train-gundetection.txt
## valid-gundetection.txt
## test-gundetection.txt

### [only once] Set checkpoint

In [0]:
# AlexeyAB 의 Darknet 같은 경우에는 아직 어떻게 하는지 모르겠음.
# google 은 연결 시간이 최대 12 시간이며 불안정하기 때문에 checkpoint 가 12시간 이내로 찍히도록 설정해 두어야 

### Training Start

In [0]:
%cd /content/gdrive/"My Drive"/darknet
%ls

# 실행권한 부여
!chmod u+x ./darknet

#----------------------------------------

# pretrained model 적용시 : downloaded weight
#!./darknet detector train data/customdata/gundata/obj-gundetection.data cfg/yolov2-tiny-voc-gundetection.cfg weights/yolov2-tiny-voc.conv.13 -map  -dont_show


# pretrained model 적용시 : last weight
# !./darknet detector train data/customdata/gundata/obj-gundetection.data cfg/yolov2-tiny-voc-gundetection.cfg backup/yolov2-tiny-voc-gundetection_last.weights -map -dont_show

# pretrained model 적용 안할시
!./darknet detector train data/customdata/gundata/obj-gundetection.data cfg/yolov2-tiny-voc-gundetection.cfg -map -dont_show


## Training model : yolo v1 *No Support*

### [only once] Change .cfg file

In [0]:
# darknet/cfg/yolov1/yolo.cfg 의 사본을 만들고, 이름을 변경
# darknet/cfg/yolov1/yolov1-gundetection.cfg



# yolov1-gundetection.cfg

'''
[net]
batch=32
subdivisions=2
height=448
width=448
channels=3
momentum=0.9
decay=0.0005
saturation=1.5
exposure=1.5
hue=.1
'''


'''
[connected]
output=784
activation=linear

[detection]
classes=1
coords=4
rescore=1
side=7
num=3
softmax=0
sqrt=1
jitter=.2
'''

'\n[net]\nbatch=64\nsubdivisions=2\nheight=448\nwidth=448\nchannels=3\nmomentum=0.9\ndecay=0.0005\nsaturation=1.5\nexposure=1.5\nhue=.1\n\nlearning_rate=0.0005\npolicy=steps\nsteps=200,400,600,20000,30000\nscales=2.5,2,2,.1,.1\nmax_batches = 40000\n'

### Training Start

In [0]:
%cd /content/gdrive/"My Drive"/darknet
%ls

# 실행권한 부여
!chmod u+x ./darknet

#----------------------------------------

# pretrained model 적용 안할시
# Or just train with -map flag
!./darknet detector train data/customdata/gundata/obj-gundetection.data cfg/yolov1/yolov1-gundetection.cfg -map -dont_show

## Training model : yolo v3

### [only once] Change .cfg file

In [0]:
# # https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects 참고

### [only once] Get pretrained weight

In [0]:
%cd /content/gdrive/"My Drive"/darknet
!./darknet partial cfg/yolov3-gundetection.cfg yolov3.weights weights/yolov3.conv.74 74

### Training Start

In [0]:
%cd /content/gdrive/"My Drive"/darknet
%ls

# 실행권한 부여
!chmod u+x ./darknet

#----------------------------------------

# pretrained model 적용시 : downloaded weight
!./darknet detector train data/customdata/gundata/obj-gundetection.data cfg/yolov3-gundetection.cfg weights/yolov3.conv.74 -map  -dont_show


# pretrained model 적용시 : last weight
# !./darknet detector train data/customdata/gundata/obj-gundetection.data cfg/yolov3-gundetection.cfg backup/yolov3-gundetection_last.weights -map -dont_show

# pretrained model 적용 안할시
# !./darknet detector train data/customdata/gundata/obj-gundetection.data cfg/yolov3-gundetection.cfg -map -dont_show


# Test Custom Dataset

## Test model : tiny-yolo v2

### Change .cfg

In [0]:
# testing 아래와 training 위의 주석을 풀어 주는 것이 중요.

'''
[net] 
# Testing 
batch=1 
subdivisions=1 
# Training 
# batch=64
# subdivisions=16 
width=608 
height=608
'''

### Test one image

In [0]:
# 디렉터리 이동
%cd /content/gdrive/"My Drive"/darknet
!rm predictions.jpg

# Example of custom object detection: darknet.exe detector test data/obj.data yolo-obj.cfg yolo-obj_8000.weights
# data/customdata/gundata/images_and_labels/armas \(3000\).jpg
# ./darknet detector test cfg/obj.data cfg/yolov3-obj.cfg yolov3-_4000.weights data/.mp4

# 실행 명령
# reference : https://dhhwang89.tistory.com/105
# bash example : !./darknet detector test cfg/coco.data cfg/yolov3.cfg yolov3.weights data/dog.jpg
# image path : data/customdata/gundata/images_and_labels/armas (2998).jpg
!./darknet detector test data/customdata/gundata/obj-gundetection.data cfg/yolov2-tiny-voc-gundetection.cfg backup/yolov2-tiny-voc-gundetection_last.weights -dont_show

# predictions.jpg 저장됨.

/content/gdrive/My Drive/darknet
 CUDA-version: 10000 (10010), GPU count: 1  
 OpenCV version: 3.2.0
net.optimized_memory = 0 
batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256       3 x 3/ 1     26 x  26 x 128 ->   26 x  26 x 256 0.399 BF
   9 max                2x 2/ 2     26 x  26 x 

### Test list of images - JSON

In [0]:
# make directory


# darknet/results/gundetection/tiny-yolo-v2/

In [0]:
# get list of weight file

%cd /content/gdrive/"My Drive"/darknet

In [0]:
# 디렉터리 이동
%cd /content/gdrive/"My Drive"/darknet

# To process a list of images data/train.txt and save results of detection to result.json file use:
!./darknet detector test data/customdata/gundata/obj-gundetection.data cfg/yolov2-tiny-voc-gundetection.cfg backup/yolov2-tiny-voc-gundetection_last.weights -ext_output -dont_show -out results/gundetection/tiny-yolo-v2/result_last.json < data/customdata/gundata/test-gundetection.txt

### Evaluate Test score

## Test model : yolo v3

### Change .cfg

In [0]:
# yolov3-gundetection.cfg

'''
[net]
# Testing
batch=1
subdivisions=1
# Training
# batch=64
# subdivisions=16
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1
'''

'\n[net]\n# Testing\nbatch=1\nsubdivisions=1\n# Training\n# batch=64\n# subdivisions=16\nwidth=416\nheight=416\nchannels=3\nmomentum=0.9\ndecay=0.0005\nangle=0\nsaturation = 1.5\nexposure = 1.5\nhue=.1\n'

### Test one image

In [0]:
# 디렉터리 이동
%cd /content/gdrive/"My Drive"/darknet
!rm predictions.jpg

# Example of custom object detection: darknet.exe detector test data/obj.data yolo-obj.cfg yolo-obj_8000.weights
# data/customdata/gundata/images_and_labels/armas \(3000\).jpg
# ./darknet detector test cfg/obj.data cfg/yolov3-obj.cfg yolov3-_4000.weights data/.mp4

# 실행 명령
# reference : https://dhhwang89.tistory.com/105
# bash example : !./darknet detector test cfg/coco.data cfg/yolov3.cfg yolov3.weights data/dog.jpg
# image path : data/customdata/gundata/images_and_labels/armas (2998).jpg
!./darknet detector test data/customdata/gundata/obj-gundetection.data cfg/yolov3-gundetection.cfg backup/yolov3-gundetection_last.weights -dont_show

# predictions.jpg 저장됨.